In [ ]:
import pandas as pd
import json
import random
import re
from pathlib import Path
from collections import defaultdict

INPUT_CSV = r"# === NOTE: Replace with local path ==="
OUTPUT_DIR = Path("./hybrid_training_data")
TRAIN_SPLIT = 0.85

OUTPUT_DIR.mkdir(exist_ok=True)


RESPONSE_TEMPLATES = {
    "excelente": [
        "{profesor} es excelente opción ({cal}/10). {destacado}. {consejo_positivo}",
        "Te recomiendo {profesor} ({cal}/10). {destacado}. {consejo_positivo}",
        "Con {profesor} no te equivocas, tiene {cal}/10. {destacado}. {consejo_positivo}"
    ],
    "bueno": [
        "{profesor} es buena opción ({cal}/10). {destacado}, aunque {limitacion}.",
        "{profesor} tiene {cal}/10. Los estudiantes mencionan {destacado}, pero {limitacion}.",
        "Con {profesor} ({cal}/10) ten en cuenta: {destacado}, sin embargo {limitacion}."
    ],
    "regular": [
        "{profesor} tiene {cal}/10. Las opiniones varían: {destacado}, aunque {limitacion}.",
        "Con {profesor} ({cal}/10) las experiencias son mixtas. {destacado}, pero {limitacion}.",
        "{profesor} es opción media ({cal}/10). {destacado}, aunque varios comentan {limitacion}."
    ],
    "dificil": [
        "{profesor} es exigente ({cal}/10). Ten cuidado: {limitacion}. {consejo_negativo}",
        "Con {profesor} ({cal}/10) prepárate bien: {limitacion}. {consejo_negativo}",
        "{profesor} tiene {cal}/10. Los estudiantes advierten: {limitacion}. {consejo_negativo}"
    ],
    "sin_calificacion": [
        "Sobre {profesor}: {comentario_resumido}.",
        "{profesor} - Los estudiantes comentan que {comentario_resumido}.",
        "Con {profesor}, ten en cuenta: {comentario_resumido}."
    ]
}

CONSEJOS = {
    "positivo": [
        "Muy recomendado si buscas calidad",
        "Es de las mejores opciones",
        "Buena elección para aprender bien"
    ],
    "negativo": [
        "Tómalo solo si te comprometes a estudiar",
        "Considera otras opciones si buscas algo más tranquilo",
        "Requiere dedicación constante"
    ]
}


QUESTION_TEMPLATES = [
    "¿Qué opinas del profesor {nombre}?",
    "¿Cómo es {nombre}?",
    "¿Recomiendas a {nombre}?",
    "Cuéntame sobre {nombre}",
    "¿Qué tal {nombre}?",
    "Información de {nombre}",
    "¿Vale la pena tomar con {nombre}?"
]

def extraer_calificacion(comentarios):
    """Extrae calificación del formato 'Calificación: X.X/10'"""
    match = re.search(r'Calificación:\s*(\d+\.?\d*)/10', comentarios)
    if match:
        return float(match.group(1))
    return None

def extraer_tags(comentarios):
    """Extrae tags en MAYÚSCULAS del comentario"""
    
    if " - " in comentarios:
        tags_text = comentarios.split(" - ", 1)[1]
        tags = [t.strip() for t in tags_text.split(",")]
        return [t for t in tags if t.isupper() or t.replace(" ", "").isupper()][:5]
    return []

def extraer_comentario_texto(comentarios):
    """Extrae texto descriptivo (sin calificación ni tags)"""
    
    texto = re.sub(r'Calificación:\s*\d+\.?\d*/10\s*-?\s*', '', comentarios)
    
    
    if " - " in texto and any(c.isupper() for c in texto.split(" - ")[1]):
        texto = texto.split(" - ")[0]
    
    
    if len(texto) > 300:
        texto = texto[:297] + "..."
    
    return texto.strip()

def clasificar_profesor(calificacion):
    """Clasifica profesor según calificación"""
    if calificacion is None:
        return "sin_calificacion"
    if calificacion >= 8.5:
        return "excelente"
    elif calificacion >= 7.0:
        return "bueno"
    elif calificacion >= 5.0:
        return "regular"
    else:
        return "dificil"

def resumir_tags(tags):
    """Resume tags en texto natural"""
    positivos = ["CLASES EXCELENTES", "BRINDA APOYO", "INSPIRACIONAL", "MUY CÓMICO", 
                 "DA BUENA RETROALIMENTACIÓN", "RESPETADO POR LOS ESTUDIANTES"]
    negativos = ["CALIFICA DURO", "MUCHAS TAREAS", "LOS EXÁMENES SON DIFÍCILES",
                 "ASISTENCIA OBLIGATORIA", "LAS CLASES SON LARGAS", "BARCO"]
    
    
    if not tags:
        return "da buenas clases", "es algo exigente"
    
    tags_positivos = [t for t in tags if t in positivos]
    tags_negativos = [t for t in tags if t in negativos]
    
    
    if not tags_positivos and not tags_negativos:
        
        primer_tag = tags[0].lower()
        return primer_tag, "tiene sus particularidades"
    
    destacado = tags_positivos[0].lower() if tags_positivos else tags[0].lower()
    limitacion = tags_negativos[0].lower() if tags_negativos else "es algo exigente"
    
    return destacado, limitacion

def generar_respuesta_natural(profesor_data):
    """Genera respuesta natural basada en datos del profesor"""
    nombre = profesor_data['nombre']
    calificacion = profesor_data['calificacion']
    tags = profesor_data['tags']
    comentario = profesor_data['comentario']
    
    categoria = clasificar_profesor(calificacion)
    template = random.choice(RESPONSE_TEMPLATES[categoria])
    
    if categoria == "sin_calificacion":
        
        return template.format(
            profesor=nombre,
            comentario_resumido=comentario[:150]
        )
    
    destacado, limitacion = resumir_tags(tags)
    consejo_key = "positivo" if categoria in ["excelente", "bueno"] else "negativo"
    consejo = random.choice(CONSEJOS[consejo_key])
    
    respuesta = template.format(
        profesor=nombre,
        cal=f"{calificacion:.1f}" if calificacion else "N/A",
        destacado=destacado,
        limitacion=limitacion if categoria in ["bueno", "regular"] else "",
        consejo_positivo=consejo if categoria in ["excelente", "bueno"] else "",
        consejo_negativo=consejo if categoria in ["regular", "dificil"] else ""
    )
    
    return respuesta.strip()

def crear_contexto_rag(profesor_data):
    """Simula contexto que vendría de RAG"""
    lineas = []
    
    if profesor_data['calificacion']:
        lineas.append(f"- Calificación: {profesor_data['calificacion']:.1f}/10")
    
    if profesor_data['tags']:
        lineas.append(f"- Tags: {', '.join(profesor_data['tags'][:3])}")
    
    if profesor_data['comentario']:
        lineas.append(f"- Comentario: {profesor_data['comentario'][:150]}")
    
    if profesor_data['materia']:
        lineas.append(f"- Materia: {profesor_data['materia']}")
    
    return "Contexto:\n" + "\n".join(lineas)

def crear_ejemplo(profesor_data):
    """Crea ejemplo de entrenamiento completo"""
    contexto = crear_contexto_rag(profesor_data)
    pregunta_template = random.choice(QUESTION_TEMPLATES)
    pregunta = pregunta_template.format(nombre=profesor_data['nombre'])
    
    user_message = f"{contexto}\n\nPregunta: {pregunta}"
    assistant_message = generar_respuesta_natural(profesor_data)
    
    return {
        "messages": [
            {
                "role": "system",
                "content": "Eres ChatCUCEI, un asistente amigable que ayuda a estudiantes de CUCEI a elegir profesores. Recibirás contexto con información real de reseñas y debes responder de forma natural y útil."
            },
            {
                "role": "user",
                "content": user_message
            },
            {
                "role": "assistant",
                "content": assistant_message
            }
        ]
    }

def main():
    print("Generando dataset híbrido desde CSV real...")
    print(f"Leyendo: {INPUT_CSV}\n")
    
    
    df = pd.read_csv(INPUT_CSV, encoding='utf-8')
    print(f"Total reseñas: {len(df)}")
    
    
    profesores_dict = defaultdict(lambda: {
        'nombre': '',
        'calificaciones': [],
        'tags': set(),
        'comentarios': [],
        'materias': set()
    })
    
    for _, row in df.iterrows():
        nombre = str(row['PROFESOR']).strip()
        comentarios = str(row['COMENTARIOS'])
        
        
        cal = extraer_calificacion(comentarios)
        tags = extraer_tags(comentarios)
        texto = extraer_comentario_texto(comentarios)
        materia = str(row['MATERIA']) if pd.notna(row['MATERIA']) else ""
        
        profesores_dict[nombre]['nombre'] = nombre
        if cal:
            profesores_dict[nombre]['calificaciones'].append(cal)
        profesores_dict[nombre]['tags'].update(tags)
        if texto and len(texto) > 10:
            profesores_dict[nombre]['comentarios'].append(texto)
        if materia:
            profesores_dict[nombre]['materias'].add(materia)
    
    print(f"Profesores únicos: {len(profesores_dict)}\n")
    
    
    profesores_list = []
    for nombre, data in profesores_dict.items():
        
        cal_promedio = sum(data['calificaciones']) / len(data['calificaciones']) if data['calificaciones'] else None
        
        
        comentario_mejor = max(data['comentarios'], key=len) if data['comentarios'] else ""
        
        profesores_list.append({
            'nombre': nombre,
            'calificacion': cal_promedio,
            'tags': list(data['tags'])[:5],
            'comentario': comentario_mejor,
            'materia': ', '.join(list(data['materias'])[:2]) if data['materias'] else ""
        })
    
    
    print("Generando ejemplos de entrenamiento...")
    ejemplos = []
    
    for profesor in profesores_list:
        
        num_variaciones = random.randint(1, 2)
        
        for _ in range(num_variaciones):
            ejemplo = crear_ejemplo(profesor)
            ejemplos.append(ejemplo)
    
    
    random.shuffle(ejemplos)
    split_idx = int(len(ejemplos) * TRAIN_SPLIT)
    
    train_ejemplos = ejemplos[:split_idx]
    val_ejemplos = ejemplos[split_idx:]
    
    
    train_file = OUTPUT_DIR / "train.jsonl"
    val_file = OUTPUT_DIR / "validation.jsonl"
    
    with open(train_file, 'w', encoding='utf-8') as f:
        for ej in train_ejemplos:
            f.write(json.dumps(ej, ensure_ascii=False) + '\n')
    
    with open(val_file, 'w', encoding='utf-8') as f:
        for ej in val_ejemplos:
            f.write(json.dumps(ej, ensure_ascii=False) + '\n')
    
    print(f"\n{'='*70}")
    print("✅ DATASET GENERADO")
    print(f"{'='*70}")
    print(f"📚 Train: {len(train_ejemplos)} ejemplos → {train_file}")
    print(f"📖 Val: {len(val_ejemplos)} ejemplos → {val_file}")
    print(f"👨‍🏫 Profesores: {len(profesores_list)}")
    
    #
    print(f"\n{'='*70}")
    print("📝 EJEMPLO DE ENTRENAMIENTO")
    print(f"{'='*70}")
    ej = train_ejemplos[0]
    print(f"USER:\n{ej['messages'][1]['content']}\n")
    print(f"ASSISTANT:\n{ej['messages'][2]['content']}")
    print(f"{'='*70}")
    
    print(f"\n💡 SIGUIENTE PASO:")
    print(f"   1. Ejecuta finetune_phi3_hybrid.py")
    print(f"   2. Luego setup_rag_system.py")
    print(f"   3. Finalmente inference_hybrid.py para usar el sistema completo")

if __name__ == "__main__":
    main()

Generando dataset híbrido desde CSV real...
Leyendo: C:\Users\morde\Desktop\NLP-CUCEI\src\data\raw\csv_completo\evaluaciones_con_departamentos.csv

Total reseñas: 1513
Profesores únicos: 875

Generando ejemplos de entrenamiento...

✅ DATASET GENERADO
📚 Train: 1109 ejemplos → hybrid_training_data\train.jsonl
📖 Val: 196 ejemplos → hybrid_training_data\validation.jsonl
👨‍🏫 Profesores: 875

📝 EJEMPLO DE ENTRENAMIENTO
USER:
Contexto:
- Calificación: 8.0/10
- Tags: CLASES EXCELENTES, POCOS EXÁMENES, ASISTENCIA OBLIGATORIA
- Comentario: ASISTENCIA OBLIGATORIA, POCOS EXÁMENES, CLASES EXCELENTES

Pregunta: ¿Vale la pena tomar con DANIEL ESCOBAR HERNANDEZ?

ASSISTANT:
DANIEL ESCOBAR HERNANDEZ es buena opción (8.0/10). clases excelentes, aunque asistencia obligatoria.

💡 SIGUIENTE PASO:
   1. Ejecuta finetune_phi3_hybrid.py
   2. Luego setup_rag_system.py
   3. Finalmente inference_hybrid.py para usar el sistema completo
